# Google ADK + Opik Integration Cookbook

This notebook demonstrates how to integrate Google's Agent Development Kit (ADK) with Opik for comprehensive tracing and observability.

You will need:

1. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
2. An OpenAI account, for using gpt-4o model - [platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)
3. Google ADK installed and configured

This example will use:

- [google-adk](https://pypi.org/project/google-adk/) for agent development
- [opik](https://pypi.org/project/opik/) for tracing and observability
- OpenAI's gpt-4o model through LiteLLM

## Setup

Install the required packages:

In [1]:
%pip install opik google-adk litellm --upgrade

Note: you may need to restart the kernel to use updated packages.


Configure Opik for your session:

In [2]:
import opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /home/mavrick/.opik.config


Set up your OpenAI API key:

In [3]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Import Required Libraries

In [4]:
import asyncio
import datetime
from zoneinfo import ZoneInfo

from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from opik.integrations.adk import OpikTracer

## Define Tools

Let's create some simple tools for our agent to demonstrate the integration:

In [5]:
def get_weather(city: str) -> dict:
    """Get weather information for a city."""
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": "The weather in New York is sunny with a temperature of 25 °C (77 °F).",
        }
    return {"status": "error", "error_message": f"Weather info for '{city}' is unavailable."}

def get_current_time(city: str) -> dict:
    """Get current time for a city."""
    if city.lower() == "new york":
        tz = ZoneInfo("America/New_York")
        now = datetime.datetime.now(tz)
        return {
            "status": "success",
            "report": now.strftime(f"The current time in {city} is %Y-%m-%d %H:%M:%S %Z%z."),
        }
    return {"status": "error", "error_message": f"No timezone info for '{city}'."}

## Configure Opik Tracing

Set up the Opik tracer to capture all agent interactions:

In [6]:
opik_tracer = OpikTracer(
    name="adk-gpt-4o-demo",
    tags=["google-adk", "openai", "gpt-4o", "cookbook"],
    metadata={
        "environment": "development",
        "model": "gpt-4o",
        "framework": "google-adk"
    },
    project_name="adk-gpt-4o-demo",
    distributed_headers={
        "trace_id": "opik-trace-id",
        "span_id": "opik-span-id"
    }
)

## Create the LLM Agent

Initialize the Google ADK agent with OpenAI's gpt-4o model and Opik tracing:

In [7]:
# Initialize LiteLLM with OpenAI gpt-4o
llm = LiteLlm(model="openai/gpt-4o")

# Create the agent with Opik callbacks
root_agent = LlmAgent(
    name="weather_time_agent",
    model=llm,
    description="Agent for answering time & weather questions via gpt-4o",
    instruction="Answer questions about the time or weather in a city.",
    tools=[get_weather, get_current_time],
    before_agent_callback=opik_tracer.before_agent_callback,
    after_agent_callback=opik_tracer.after_agent_callback,
    before_model_callback=opik_tracer.before_model_callback,
    after_model_callback=opik_tracer.after_model_callback,
    before_tool_callback=opik_tracer.before_tool_callback,
    after_tool_callback=opik_tracer.after_tool_callback,
)

## Setup Session and Runner

Configure the session service and runner for managing conversations:

In [8]:
session_service = InMemorySessionService()
runner = Runner(
    agent=root_agent,
    app_name="weather_app",
    session_service=session_service,
)

## Helper Functions

Define utility functions for session management and agent interaction:

In [9]:
async def setup_session():
    """Create a new session for the conversation."""
    sess = await session_service.create_session(
        app_name="weather_app",
        user_id="user_123",
        session_id="session_abc"
    )
    return sess.id

async def call_agent_async(user_msg: str, session_id: str):
    """Send a message to the agent and get the response."""
    content = types.Content(role="user", parts=[types.Part(text=user_msg)])
    async for event in runner.run_async(user_id="user_123", session_id=session_id, new_message=content):
        if event.is_final_response():
            print("Assistant:", event.content.parts[0].text)

## Demo: Interacting with the Agent

Now let's run some example interactions with our agent. All interactions will be automatically traced in Opik:

In [10]:
# Create a session
session_id = await setup_session()
print(f"Created session: {session_id}")

Created session: session_abc


In [11]:
# Ask about weather
print("User: What's the weather in New York?")
await call_agent_async("What's the weather in New York?", session_id)

User: What's the weather in New York?


OPIK: Started logging traces to the "adk-gpt-4o-demo" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197a718-83d0-7a32-b6e9-a3e8dd12ed81&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Assistant: The current weather in New York is sunny with a temperature of 25 °C (77 °F).


In [12]:
# Ask about time
print("\nUser: And what time is it now?")
await call_agent_async("And what time is it now?", session_id)


User: And what time is it now?
Assistant: The current time in New York is 8:38 AM on June 25th, 2025.


The trace can now be viewed in the UI:

![Google adk  Cookbook](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/google_adk_integration.png)

## Viewing Traces in Opik

After running the demo, you can view the traces in your Opik dashboard:

1. Go to [comet.com](https://comet.com) and log into your account
2. Navigate to the Opik section
3. Look for the project named "adk-gpt-4o-demo"
4. You'll see traces for each interaction, including:
   - Agent calls
   - Model invocations
   - Tool executions
   - Response generation

The traces will show the complete flow of each conversation, making it easy to debug and optimize your agent's performance.

## Next Steps

This notebook demonstrated the basic integration between Google ADK and Opik. You can extend this by:

- Adding more sophisticated tools and functions
- Implementing custom evaluation metrics
- Using different LLM models through LiteLLM
- Adding more complex conversation flows
- Implementing custom tracing strategies

For more information:
- [Google ADK Documentation](https://cloud.google.com/agent-development-kit)
- [Opik Documentation](https://www.comet.com/docs/opik/)
- [LiteLLM Documentation](https://docs.litellm.ai/)